In [1]:
# replace pracs csv load with SQL

# Set dates of baseline and follow-up periods
d4 = '2019-03-01' # month after end of follow-up period
d3 = '2018-09-01' # follow-up start
d2 = '2018-03-01' # month after end of baseline period
d1 = '2017-09-01' # baseline start

# Import dataset from BigQuery
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

q = '''SELECT * FROM ebmdatalab.measures.practice_data_ktt9_cephalosporins
WHERE EXTRACT (YEAR from month)  >= 2017
'''
df1 = pd.read_gbq(q, GBQ_PROJECT_ID, dialect='standard',verbose=False)

        ## note: parsing dates is quite memory-intensive, make sure not too many programmes running

df1["month"] = pd.to_datetime(df1.month)

df1.head() # this gives the first few rows of data

,numerator,denominator,practice_id,pct_id,month,calc_value,percentile
0,9,199,A81001,00K,2017-04-01,0.045226,0.121870
1,10,165,A81001,00K,2017-07-01,0.060606,0.206994
2,7,158,A81001,00K,2017-08-01,0.044304,0.089961
3,11,206,A81001,00K,2017-09-01,0.053398,0.155435
4,14,256,A81001,00K,2017-01-01,0.054688,0.248203


In [2]:
### filter out the baseline and follow-up periods
import datetime

conditions = [
    (df1['month']  >= d4), # after follow-up period
    (df1['month']  >= d3), # follow-up
    (df1['month']  >= d2), # mid
    (df1['month']  >= d1), # baseline
    (df1['month']  < d1)] # before

choices = ['after', 'follow-up', 'mid', 'baseline','before']
df1['period'] = np.select(conditions, choices, default='0')

df1.head()

,numerator,denominator,practice_id,pct_id,month,calc_value,percentile,period
0,9,199,A81001,00K,2017-04-01,0.045226,0.121870,before
1,10,165,A81001,00K,2017-07-01,0.060606,0.206994,before
2,7,158,A81001,00K,2017-08-01,0.044304,0.089961,before
3,11,206,A81001,00K,2017-09-01,0.053398,0.155435,baseline
4,14,256,A81001,00K,2017-01-01,0.054688,0.248203,before


In [3]:
### import practices eligible for study
q = '''
SELECT 
prac.code AS practice_id, 
prac.name,
ccg_id, 
total_list_size, 
(male_25_34+ male_35_44+ male_45_54+ male_55_64+ female_25_34+ female_35_44+ female_45_54+ female_55_64)/total_list_size AS percent_25_64,
CAST (open_date AS DATE) AS open_date, 
--CASE WHEN d.code IS NULL THEN 0 ELSE 1 END as dispensing,
sum(items) as items,
1000*IEEE_divide(sum(items),total_list_size) AS items_per_thou
      
from ebmdatalab.hscic.practices prac
INNER JOIN ebmdatalab.hscic.practice_statistics stat ON prac.code = stat.practice AND DATE(stat.month) = '2018-02-01'   -- latest month
LEFT JOIN ebmdatalab.hscic.prescribing_2018_02 p ON prac.code = p.practice -- latest month's prescribing data 
--LEFT JOIN  ebmdatalab.hscic.practices_mem m ON prac.code = m.practice AND SUBSTR(CAST(m.join_date AS STRING),1,4) > '2016' AND m.ccg_change_flag = 1
--LEFT JOIN ebmdatalab.bsa.dispensing_practices_jan2017 d ON prac.code= d.code AND dispensing_patients > 0

WHERE 
prac.ccg_id NOT IN ('99P')  -- exclude any CCGs involved in preliminary testing
AND prac.code NOT IN ('')       -- exclude any practices involved in preliminary testing -- replace with [list of practices to exclude]
AND prac.setting = 4            -- include standard practices only
AND prac.status_code = 'A'      -- active status (exclude dormant and closed)
AND CAST (open_date AS DATE) < '2017-09-01' -- only include practices opened prior to baseline period (Sep-Feb)
AND stat.total_list_size >= 1000    -- minimum list size (at latest quarter)
AND SUBSTR(ccg_id,1,1) NOT BETWEEN 'A' AND 'Z'      -- this will exclude any practices belonging to NHS Trusts rather than CCGs as they are not standard practices

GROUP BY stat.month, practice_id, percent_25_64, name, ccg_id, total_list_size, open_date

HAVING items >= 1000 
ORDER BY items asc, total_list_size asc
'''
#prac = pd.read_csv('eligible_pracs_test.csv')
prac = pd.io.gbq.read_gbq(q, GBQ_PROJECT_ID, dialect='standard',verbose=False)

prac.head()

,practice_id,name,ccg_id,total_list_size,percent_25_64,open_date,items,items_per_thou
0,F86704,PAULZ SURGERY,08N,1861,0.584095,1993-09-01,1043,560.451370
1,M85085,MODALITY ANN JONES FAMILY HEALTH CENTRE,05L,1965,0.546056,1974-04-01,1045,531.806616
2,F84660,DR CM PATEL'S SURGERY,08M,2172,0.707643,1974-04-01,1051,483.885820
3,K83020,RILLWOOD MEDICAL CENTRE,04G,3322,0.548465,1985-05-01,1090,328.115593
4,A88603,THE PARK SURGERY,00N,2913,0.539993,1974-04-01,1100,377.617576


In [6]:
## join data with eligible practices
# take columns of interest from df
df2 = df1[["practice_id","period", "month", "numerator","denominator"]]
df2 = df2.set_index(["practice_id","period", "month"])
dfm = df2.reset_index()
dfm = prac.merge(dfm, how='left', on='practice_id')
# take columns of interest from df
dfm = dfm[["practice_id","period","percent_25_64","month", "numerator","denominator"]].loc[(dfm.period=="baseline")]# | (dfm.period=="follow-up")]
dfm.head(20)

,practice_id,period,percent_25_64,month,numerator,denominator
1,F86704,baseline,0.584095,2018-02-01,2,32
2,F86704,baseline,0.584095,2017-11-01,4,27
3,F86704,baseline,0.584095,2017-12-01,2,31
4,F86704,baseline,0.584095,2017-09-01,2,30
8,F86704,baseline,0.584095,2018-01-01,6,30
10,F86704,baseline,0.584095,2017-10-01,6,40
14,M85085,baseline,0.546056,2017-12-01,2,26
15,M85085,baseline,0.546056,2017-10-01,5,33
16,M85085,baseline,0.546056,2017-11-01,2,32
17,M85085,baseline,0.546056,2017-09-01,2,33


In [7]:
### aggregate data over 6-month periods ( we will want to calculate the change between each)

# Perform groupby aggregation
agg_6m = dfm.groupby(["practice_id","percent_25_64", "period"]).sum() 

### filter out measures not meeting threshold values (i.e. less than 10 items prescribed on average per month)
agg_6m_f = agg_6m.loc[(agg_6m.denominator>60)] # filtering only on sum should suffice

agg_6m_f = agg_6m_f.reset_index().loc[(agg_6m_f.reset_index().percent_25_64 < 0.85) & (agg_6m_f.reset_index().percent_25_64 > 0.1)]


### calculate aggregated measure values
agg_6m_f["calc_value"] = agg_6m_f.numerator / agg_6m_f.denominator

#agg_6m_f.head()
print (agg_6m.denominator.loc[(agg_6m.denominator<=60)].count(), 'practices excluded for prescribing <60 total antibiotics in 6 months.')

print (agg_6m.reset_index().practice_id.loc[(agg_6m.reset_index().percent_25_64>=0.85)].count()+
       agg_6m.reset_index().practice_id.loc[(agg_6m.reset_index().percent_25_64<=0.1)].count(),
       'practices excluded for having unusually high or low adult age population (likely homeless or care home services.')

print( len(agg_6m_f), 'practices included.')


0 practices excluded for prescribing <60 total antibiotics in 6 months.
8 practices excluded for having unusually high or low adult age population (likely homeless or care home services.
7004 practices included.


In [8]:
# extract calc_value column and unstack years
agg_6m_f = agg_6m_f.loc[~agg_6m_f.calc_value.isnull()] # exclude the extra row at bottom
dfx = agg_6m_f.reset_index()
dfx = dfx[["practice_id","calc_value","denominator"]]

dfx.columns.values[1] = 'baseline'

dfx.head(10)

,practice_id,baseline,denominator
0,A81001,0.046563,1353
1,A81002,0.085483,6785
2,A81004,0.081362,3171
3,A81005,0.110249,2449
4,A81006,0.079103,4728
5,A81007,0.063672,2827
6,A81009,0.079457,2945
7,A81011,0.071679,3711
8,A81012,0.090154,1686
9,A81013,0.093242,1598


In [9]:
### calculate percentile for each practice for single measure during baseline period and flag those which are in worst 20%

df3 = dfx.copy()
df3["baseline_ranking"] = df3["baseline"].rank(method='min', pct=True)
df3["baseline_worst20"] = df3["baseline_ranking"] >= 0.8

#df3.to_csv('cephalosporin_change_201617.csv')
df3.head(10)

,practice_id,baseline,denominator,baseline_ranking,baseline_worst20
0,A81001,0.046563,1353,0.103655,False
1,A81002,0.085483,6785,0.581525,False
2,A81004,0.081362,3171,0.529697,False
3,A81005,0.110249,2449,0.841662,True
4,A81006,0.079103,4728,0.496859,False
5,A81007,0.063672,2827,0.280697,False
6,A81009,0.079457,2945,0.501571,False
7,A81011,0.071679,3711,0.389063,False
8,A81012,0.090154,1686,0.639206,False
9,A81013,0.093242,1598,0.676756,False


In [10]:
### lookup practice ccg to use for allocation
df4 = df3.loc[df3.baseline_worst20==True]
df4 = df4[["practice_id","baseline","baseline_ranking"]]
df5 = df4.merge(prac, how='left', on='practice_id')

df5 = df5[["practice_id","ccg_id","baseline",'baseline_ranking']]

### repeat for the non-allocated practices (those outside worst 20%, maybe needed for controls for clicks)
others = df3.loc[df3.baseline_worst20==False]
others = others[["practice_id","baseline","baseline_ranking"]]
others = others.merge(prac, how='left', on='practice_id')

others = others[["practice_id","ccg_id","baseline",'baseline_ranking']]

df5.head()

,practice_id,ccg_id,baseline,baseline_ranking
0,A81005,00M,0.110249,0.841662
1,A81016,00M,0.113095,0.861936
2,A81022,00M,0.106650,0.812678
3,A81030,00M,0.129118,0.933181
4,A81040,00K,0.106930,0.815534


In [12]:
### allocate bottom 20% practices to intervention and control groups 

# set seeds for random number generation to ensure repeatable
seed1 = 321
seed2 = 213
seed3 = 312

import random as rd

np.random.seed(seed1)
df5['rand_num'] = np.random.rand(len(df5))
df5["allocation_ranking"] = df5.groupby('ccg_id').rand_num.rank()

df5["allocation_code"]= df5.allocation_ranking.mod(2)
df5 = df5.sort_values(by=['ccg_id','allocation_ranking']) 

#assign each ccg to a random start point
ccgs = df5.loc[df5.allocation_ranking ==1].reset_index()
ccgs = ccgs[['ccg_id']]
np.random.seed(seed2)
ccgs['start_int'] = np.random.randint(1,3, size=len(ccgs)) 
np.random.seed(seed3)
ccgs['start_int2'] = np.random.randint(1,3, size=len(ccgs)) 

print (ccgs.ccg_id.count(), 'CCGs are included in the intervention.')

# join tables back together
df6 = df5.merge(ccgs, how='left', on='ccg_id')

#create final allocation groups
df6['final_allocation'] = np.where(df6['start_int']==2,df6.allocation_code,1-df6.allocation_code)
df6['allocation'] = np.where(df6['final_allocation']==0,'con','I')
#df6.sort_values(by=['ccg_id', 'rand_num']) 
print (df6.loc[df6.allocation=="I"].practice_id.count(), 'practices have been assigned to the intervention group.')
print (df6.loc[df6.allocation=="con"].practice_id.count(), 'practices have been assigned to the control group.')

170 CCGs are included in the intervention.
703 practices have been assigned to the intervention group.
698 practices have been assigned to the control group.


In [13]:
### Allocate Intervention practices into groups A and B
# stratify by ccg and baseline ranking

df6["ranking2"] = df6.groupby(['ccg_id','allocation']).baseline_ranking.rank()
df6["allocation_code2"]= df6.ranking2.mod(2)
df6["group_ab_2"]= np.where(df6['start_int2']==2,df6.allocation_code2,1-df6.allocation_code2)
df6['group_ab_3'] = np.where(df6['group_ab_2']==0,'A','B') 
df6['group_ab'] = np.where(df6['allocation']=='con','con',df6.group_ab_3) 
     
df7 = df6[['practice_id','ccg_id','baseline','baseline_ranking','allocation','group_ab']].sort_values(by=['ccg_id', 'practice_id'])

print (df7.loc[df7.group_ab=="A"].practice_id.count(), 'practices have been assigned to Intervention A.')
print (df7.loc[df7.group_ab=="A"].baseline.mean(), 'is the mean baseline measure for practices in Intervention A.')
print (df7.loc[df7.group_ab=="B"].practice_id.count(), 'practices have been assigned to Intervention B.')
print (df7.loc[df7.group_ab=="B"].baseline.mean(), 'is the mean baseline measure for practices in Intervention B.')

356 practices have been assigned to Intervention A.
0.12768878394893346 is the mean baseline measure for practices in Intervention A.
347 practices have been assigned to Intervention B.
0.12632462463913716 is the mean baseline measure for practices in Intervention B.


In [12]:
df7.to_csv('practice_allocations.csv') 
#df7.groupby(df7.group_ab).practice_id.count()